<a href="https://colab.research.google.com/github/singhsukhendra/MovieGenrePrediction/blob/master/SplitComplexLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 229 Final Project Notebook 

### Overview

Implementation of LeNet-5 and similar architectures to evaluate complex- and split-complex-valued neural networks. Network is implemented in TensorFlow, and wrappers for complex and split-complex valued implementations are given in layersHDA.py ("layers high-dimensional algebra"). 

### High-dimensional Algebras

To implement layers with complex- and split-complex-valued weights, we can store the weights for the real/imaginary channels separately and use weight-sharing between operations. This is implemented with wrapper files for TensorFlow commands. 

### References
Code implemented here was based on open source code available at: https://github.com/sujaybabruwad/LeNet-in-Tensorflow. The functions to extract CIFAR-10, CIFAR-100, and SVHN were based on code available at: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py.


In [0]:
! mkdir ExperimentData

In [0]:
# Front matter: load libraries needed for training
import numpy as np
import tensorflow as tf
from utils import *

import os

%reload_ext autoreload
%autoreload 2

In [0]:
# Runfile preliminaries
networkSizes = ['Baseline', 'Wide','Deep']
networkTypes = ['Real', 'Complex','SplitComplex']

# MNIST Training

The cells below load MNIST data and execute run files for the various network architectures. Each iteration saves loss/accuracy curves for the training, validation, and test sets, and the model at the end of training. Models are trained for 20 epochs with a batch size of 200. 

In [4]:
# Initialize MNIST Data

dataset = 'MNIST' #set book-keeping parameter

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", reshape=False)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels

# Zero-pad MNIST images to be 32x32
X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_validation = np.pad(X_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(X_train[0].shape))

W0825 05:01:43.526800 139670556764032 deprecation.py:323] From <ipython-input-4-8cad159e7a6b>:6: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0825 05:01:43.529598 139670556764032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0825 05:01:43.534197 139670556764032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


W0825 05:01:44.614074 139670556764032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0825 05:01:45.160444 139670556764032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Updated Image Shape: (32, 32, 1)


In [0]:
# Set training parameters
EPOCHS = 40
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 

In [6]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), 
                 dataset, BATCH_SIZE, EPOCHS, num_classes = 10, networkSize = networkSize, networkType = networkType, alpha = 1e-3)

W0825 05:02:10.630701 139670556764032 deprecation_wrapper.py:119] From /content/utils.py:35: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0825 05:02:10.648162 139670556764032 deprecation_wrapper.py:119] From /content/layersHDA.py:71: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0825 05:02:10.676654 139670556764032 deprecation_wrapper.py:119] From /content/layersHDA.py:118: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0825 05:02:10.731919 139670556764032 deprecation.py:323] From /content/utils.py:49: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W0825 05:02:10.757083 139670556764032 deprecation_wrapper

Training...

EPOCH 1 ...
Training Accuracy = 0.960
Validation Accuracy = 0.950

EPOCH 2 ...
Training Accuracy = 0.960
Validation Accuracy = 0.972

EPOCH 3 ...
Training Accuracy = 0.985
Validation Accuracy = 0.980

EPOCH 4 ...
Training Accuracy = 0.975
Validation Accuracy = 0.980

EPOCH 5 ...
Training Accuracy = 0.990
Validation Accuracy = 0.982

EPOCH 6 ...
Training Accuracy = 0.985
Validation Accuracy = 0.984

EPOCH 7 ...
Training Accuracy = 0.980
Validation Accuracy = 0.984

EPOCH 8 ...
Training Accuracy = 0.995
Validation Accuracy = 0.986

EPOCH 9 ...
Training Accuracy = 0.995
Validation Accuracy = 0.986

EPOCH 10 ...
Training Accuracy = 0.995
Validation Accuracy = 0.987

EPOCH 11 ...
Training Accuracy = 0.985
Validation Accuracy = 0.988

EPOCH 12 ...
Training Accuracy = 0.995
Validation Accuracy = 0.988

EPOCH 13 ...
Training Accuracy = 1.000
Validation Accuracy = 0.988

EPOCH 14 ...
Training Accuracy = 0.975
Validation Accuracy = 0.989

EPOCH 15 ...
Training Accuracy = 1.000
Valid

W0825 05:04:19.497627 139670556764032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


EPOCH 40 ...
Training Accuracy = 0.995
Validation Accuracy = 0.990

Model saved
Test Accuracy = 0.989
Training...

EPOCH 1 ...
Training Accuracy = 0.945
Validation Accuracy = 0.962

EPOCH 2 ...
Training Accuracy = 0.980
Validation Accuracy = 0.975

EPOCH 3 ...
Training Accuracy = 0.980
Validation Accuracy = 0.982

EPOCH 4 ...
Training Accuracy = 0.995
Validation Accuracy = 0.983

EPOCH 5 ...
Training Accuracy = 0.985
Validation Accuracy = 0.986

EPOCH 6 ...
Training Accuracy = 1.000
Validation Accuracy = 0.985

EPOCH 7 ...
Training Accuracy = 0.990
Validation Accuracy = 0.987

EPOCH 8 ...
Training Accuracy = 0.995
Validation Accuracy = 0.986

EPOCH 9 ...
Training Accuracy = 1.000
Validation Accuracy = 0.986

EPOCH 10 ...
Training Accuracy = 0.990
Validation Accuracy = 0.988

EPOCH 11 ...
Training Accuracy = 0.990
Validation Accuracy = 0.987

EPOCH 12 ...
Training Accuracy = 1.000
Validation Accuracy = 0.989

EPOCH 13 ...
Training Accuracy = 1.000
Validation Accuracy = 0.990

EPOCH 14 .

# CIFAR-10 Training
Cells below load the CIFAR-10 dataset (https://www.cs.toronto.edu/~kriz/cifar.html), setup the training parameters, and make calls to train the 9 models we are interested in. 

In [7]:
# Initialize CIFAR-10 Data
# Used code from: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py
import cifar10
dataset = 'CIFAR10' #set book-keeping parameter

train_tup, test_tup = cifar10.load_data()

X_train, y_train = train_tup
X_validation = X_train[49000:,:]
y_validation = y_train[49000:]
X_train = X_train[:49000,:]
y_train = y_train[:49000]

X_test, y_test = test_tup

100.0% 170500096 / 170498071


('Succesfully downloaded', 'cifar-10-python.tar.gz', 170498071, 'bytes.')
File Extracted in Current Directory


In [8]:
# Set training parameters
EPOCHS = 100
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 
print('Number of training examples: {}'.format(NUM_TRAIN))
print('Number of validation examples: {}'.format(NUM_VAL))
print('Number of testing examples: {}'.format(NUM_TEST))

Number of training examples: 49000
Number of validation examples: 1000
Number of testing examples: 10000


In [0]:
# Runfile preliminaries
networkSizes = ['Baseline', 'Wide','Deep']
networkTypes = ['Real', 'Complex','SplitComplex']

In [11]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), dataset, 
                 BATCH_SIZE, EPOCHS, num_classes = 10, networkSize = networkSize, networkType = networkType, alpha = 1e-4)

Training...

EPOCH 1 ...
Training Accuracy = 0.320
Validation Accuracy = 0.303

EPOCH 2 ...
Training Accuracy = 0.365
Validation Accuracy = 0.362

EPOCH 3 ...
Training Accuracy = 0.395
Validation Accuracy = 0.387

EPOCH 4 ...
Training Accuracy = 0.365
Validation Accuracy = 0.404

EPOCH 5 ...
Training Accuracy = 0.435
Validation Accuracy = 0.416

EPOCH 6 ...
Training Accuracy = 0.435
Validation Accuracy = 0.423

EPOCH 7 ...
Training Accuracy = 0.465
Validation Accuracy = 0.433

EPOCH 8 ...
Training Accuracy = 0.440
Validation Accuracy = 0.447

EPOCH 9 ...
Training Accuracy = 0.425
Validation Accuracy = 0.451

EPOCH 10 ...
Training Accuracy = 0.390
Validation Accuracy = 0.453

EPOCH 11 ...
Training Accuracy = 0.405
Validation Accuracy = 0.464

EPOCH 12 ...
Training Accuracy = 0.425
Validation Accuracy = 0.470

EPOCH 13 ...
Training Accuracy = 0.415
Validation Accuracy = 0.485

EPOCH 14 ...
Training Accuracy = 0.450
Validation Accuracy = 0.487

EPOCH 15 ...
Training Accuracy = 0.495
Valid

KeyboardInterrupt: ignored

# CIFAR-10 Training (with regularization)
Cells below load the CIFAR-10 dataset (https://www.cs.toronto.edu/~kriz/cifar.html), setup the training parameters, and make calls to train the 9 models we are interested in. Here, we also employ L2 regularization.

In [0]:
# Initialize CIFAR-10 Data
# Used code from: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py
import cifar10
dataset = 'CIFAR10-L2reg' #set book-keeping parameter

train_tup, test_tup = cifar10.load_data()

X_train, y_train = train_tup
X_validation = X_train[49000:,:]
y_validation = y_train[49000:]
X_train = X_train[:49000,:]
y_train = y_train[:49000]

X_test, y_test = test_tup

In [0]:
# Set training parameters
EPOCHS = 100
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 

print('Number of training examples: {}'.format(NUM_TRAIN))
print('Number of validation examples: {}'.format(NUM_VAL))
print('Number of testing examples: {}'.format(NUM_TEST))

# Runfile preliminaries
networkSizes = ['Wide','Deep'] # ['Baseline', 'Wide','Deep']
networkTypes = ['Real', 'Complex','SplitComplex']

In [0]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), 
                 dataset, BATCH_SIZE, EPOCHS, num_classes = 10, networkSize = networkSize, 
                 networkType = networkType, alpha = 5e-4, lam = 5e-3)

# CIFAR-100 Training
Cells below load the CIFAR-100 dataset (https://www.cs.toronto.edu/~kriz/cifar.html), setup the training parameters, and make calls to train the 9 models we are interested in. (This code is nearly identical to the CIFAR-10 cells above.)

In [0]:
# Initialize CIFAR-100 Data
# Used code from: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py
import cifar100
dataset = 'CIFAR100' #set book-keeping parameter

train_tup, test_tup = cifar100.load_data()

X_train, y_train = train_tup
X_validation = X_train[49000:,:]
y_validation = y_train[49000:]
X_train = X_train[:49000,:]
y_train = y_train[:49000]

X_test, y_test = test_tup

In [0]:
# Set training parameters
EPOCHS = 50
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 
print('Number of training examples: {}'.format(NUM_TRAIN))
print('Number of validation examples: {}'.format(NUM_VAL))
print('Number of testing examples: {}'.format(NUM_TEST))

In [0]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), dataset, 
                 BATCH_SIZE, EPOCHS, num_classes = 100, networkSize = networkSize, networkType = networkType, alpha = 5e-4)

# SVHN Training
Cells below load the SVHN dataset (http://ufldl.stanford.edu/housenumbers/), setup the training parameters, and make calls to train the 9 models we are interested in. (This code is nearly identical to the CIFAR-10 cells above.)

In [13]:
# Initialize SVHN Data
# Used code from: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py
import svhn
dataset = 'SVHN' #set book-keeping parameter

train_tup, test_tup = svhn.load_data()

X_train, y_train = train_tup
X_validation = X_train[72257:]
y_validation = y_train[72257:]
X_train = X_train[:60000]
y_train = y_train[:60000]

X_test, y_test = test_tup


100.0% 182042624 / 182040794


('Succesfully downloaded', 'train_32x32.mat', 182040794, 'bytes.')


100.0% 64282624 / 64275384


('Succesfully downloaded', 'test_32x32.mat', 64275384, 'bytes.')


In [14]:
# Set training parameters
EPOCHS = 100
BATCH_SIZE = 128
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 
print('Number of training examples: {}'.format(NUM_TRAIN))
print('Number of validation examples: {}'.format(NUM_VAL))
print('Number of testing examples: {}'.format(NUM_TEST))

Number of training examples: 60000
Number of validation examples: 1000
Number of testing examples: 26032


In [20]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), dataset, 
                 BATCH_SIZE, EPOCHS, num_classes = 10, networkSize = networkSize, networkType = networkType, alpha = 1e-1)

Training...

EPOCH 1 ...
Training Accuracy = 0.333
Validation Accuracy = 0.172

EPOCH 2 ...
Training Accuracy = 0.083
Validation Accuracy = 0.135

EPOCH 3 ...
Training Accuracy = 0.250
Validation Accuracy = 0.172

EPOCH 4 ...
Training Accuracy = 0.167
Validation Accuracy = 0.172

EPOCH 5 ...
Training Accuracy = 0.156
Validation Accuracy = 0.135

EPOCH 6 ...
Training Accuracy = 0.198
Validation Accuracy = 0.172

EPOCH 7 ...
Training Accuracy = 0.177
Validation Accuracy = 0.172

EPOCH 8 ...
Training Accuracy = 0.219
Validation Accuracy = 0.172

EPOCH 9 ...
Training Accuracy = 0.115
Validation Accuracy = 0.172

EPOCH 10 ...
Training Accuracy = 0.156
Validation Accuracy = 0.172

EPOCH 11 ...
Training Accuracy = 0.188
Validation Accuracy = 0.172

EPOCH 12 ...
Training Accuracy = 0.219
Validation Accuracy = 0.172

EPOCH 13 ...
Training Accuracy = 0.240
Validation Accuracy = 0.172

EPOCH 14 ...
Training Accuracy = 0.104
Validation Accuracy = 0.172

EPOCH 15 ...
Training Accuracy = 0.156
Valid

IndexError: ignored

In [0]:
! mv  /content/ExperimentData   /content/drive/My\ Drive/OldCode/

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
